# 06 · Renormalization & Hierarchical Planning on a Ring World

We coarse-grain a fine ring world (size $N$) to macro rings (size $M$), derive coarse models $A',B'$, and compare **planning cost vs. action quality** at micro vs. macro levels. We also demo a simple **hierarchical plan**: plan in macro space first, then refine locally in micro space.

**Key operators** (from `persystems.renorm`):
- Partition map $\pi: \{0..N-1\}\to\{0..M-1\}$ → matrix $R$ (rows = macro, cols = micro). Restrict belief: $Q = Rq$.
- Right-inverse (lift) $L$ with $RL=I_M$. Lift belief: $q = LQ$.
- Coarse models: $A' = \text{block-avg}(A)$, \quad $B'^a = R B^a L$.

We measure planning **nodes expanded** and wall-time vs. $M$, and compare the chosen action against the fine-level planner at the same horizon $H$.

In [ ]:
# CI-friendly parameters
import os
CI = os.getenv("CI", "").lower() in ("1","true","yes")
N_FINE   = 12 if CI else 20        # fine ring size
H_LIST   = [1,2] if CI else [1,2,3]
M_LIST   = [3,4,6] if CI else [3,4,5,6,10]  # macro sizes to try
PRUNE    = 1e-4
print({"CI": CI, "N_FINE": N_FINE, "H_LIST": H_LIST, "M_LIST": M_LIST})

## Imports & world construction
We’ll reuse the ring world from Phase I and the coarse-graining tools from Phase II/III.

In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from persystems.gm import GenerativeModel
from persystems.renorm import contiguous_blocks_ring, coarse_grain_ringworld, restrict_belief, lift_belief
from persystems.planning import choose_action_planner

np.set_printoptions(precision=4, suppress=True)
plt.rcParams['figure.dpi'] = 120

N = N_FINE
gm = GenerativeModel.make_ring_world(N=N, A_eps=0.15, target_idx=(N//2))
qs0 = np.ones(N)/N   # uniform belief to isolate planning cost
print("Fine states:", N, "; actions:", gm.actions)

## Helper: benchmark planner at fine vs. macro levels
At macro level, we restrict the belief $q\to Q$, plan using $A',B'$, and compare chosen action to fine-level planning. Actions are the same shift set ($-1,0,+1$), so the mapping is identity.

In [ ]:
def bench_fine(gm, qs, H, prune=1e-4):
    t0 = time.perf_counter()
    a_idx, comp, Gs, diags = choose_action_planner(qs, gm.A, gm.B, gm.C, horizon=H, obs_prune_eps=prune)
    return {
        "level": "fine", "H": H, "a": a_idx,
        "nodes": diags.nodes_expanded, "pruned": diags.pruned_obs,
        "dt": time.perf_counter()-t0, "Gs": Gs
    }

def bench_macro(gm, qs, M, H, prune=1e-4):
    # Coarse-grain operators and models
    A_coarse, B_coarse, R, L, blocks = coarse_grain_ringworld(gm.A, gm.B, N=gm.A.shape[0], M=M)
    Qs = restrict_belief(R, qs)
    # plan at macro level
    t0 = time.perf_counter()
    a_idx, comp, Gs, diags = choose_action_planner(Qs, A_coarse, B_coarse, gm.C, horizon=H, obs_prune_eps=prune)
    return {
        "level": f"macro(M={M})", "H": H, "a": a_idx,
        "nodes": diags.nodes_expanded, "pruned": diags.pruned_obs,
        "dt": time.perf_counter()-t0, "Gs": Gs,
        "R": R, "L": L, "blocks": blocks
    }

## Sweep macro sizes and horizons
We compare compute (nodes, time) and action agreement with the fine planner at the same horizon $H$.

In [ ]:
rows = []
for H in H_LIST:
    ref = bench_fine(gm, qs0, H, prune=0.0)   # exact (no prune) as reference
    rows.append({**ref, "method": "fine_exact"})
    for M in M_LIST:
        rec = bench_macro(gm, qs0, M, H, prune=PRUNE)
        rec["method"] = f"macro_pruned(M={M})"
        rec["agree_with_fine"] = int(rec["a"] == ref["a"])  # 1 if same first action
        rows.append(rec)

df = pd.DataFrame(rows)
df.sort_values(["H","level"]).head(8)

### Nodes vs. macro size $M$
Coarser $M$ should reduce nodes for the same $H$, often with modest impact on the chosen action in this toy world.

In [ ]:
plt.figure(figsize=(7,4))
for H in H_LIST:
    sub = df[(df["H"]==H) & (df["level"].str.startswith("macro"))]
    # extract M from label
    Ms = sub["level"].str.extract(r"M=(\d+)").astype(int)[0]
    plt.plot(Ms, sub["nodes"], marker='o', label=f"H={H} macro")
ref_nodes = df[(df["H"]==H) & (df["level"]=="fine")]["nodes"].values[0]
    
plt.yscale('log')
plt.xlabel('macro states M')
plt.ylabel('nodes expanded (log scale)')
plt.title('Search complexity vs. macro size')
plt.legend(fontsize=8)
plt.tight_layout(); plt.show()

### Agreement with fine planner (first action)
How often the macro plan matches the fine-level first action at the same horizon $H$ (here with a uniform prior belief). In more complex setups, agreement can be improved by increasing $M$ or doing local micro refinement around the macro-suggested action/state neighborhood.

In [ ]:
agree_tbl = (
    df[df["level"].str.startswith("macro")]
    .assign(M=lambda d: d["level"].str.extract(r"M=(\d+)").astype(int))
    .groupby(["H","M"]).agg(agree_rate=("agree_with_fine","mean"), nodes_median=("nodes","median")).reset_index()
)
agree_tbl.sort_values(["H","M"]).head(10)

In [ ]:
plt.figure(figsize=(7,4))
for H in H_LIST:
    sub = agree_tbl[agree_tbl["H"]==H]
    plt.plot(sub["M"], 100*sub["agree_rate"], marker='o', label=f"H={H}")
plt.ylim(0,105)
plt.xlabel('macro states M')
plt.ylabel('agreement with fine action (%)')
plt.title('Macro plan agreement vs. M')
plt.legend(fontsize=8)
plt.tight_layout(); plt.show()

## Hierarchical plan: macro then micro
A simple two-stage planner:
1. **Macro stage**: restrict $q\to Q$, choose action $a_{macro}$ using $A',B'$, horizon $H_{macro}$.
2. **Micro stage**: lift $Q\to q$ and refine by choosing micro action with horizon $H_{micro}$ (optionally conditioned on macro suggestion).

On this symmetric ring, actions map 1:1 across levels, so we just compare the chosen actions and compute the overall compute savings (nodes).

In [ ]:
def hierarchical_plan(gm, qs, M, H_macro=2, H_micro=1, prune=1e-4):
    A_coarse, B_coarse, R, L, blocks = coarse_grain_ringworld(gm.A, gm.B, N=gm.A.shape[0], M=M)
    Qs = restrict_belief(R, qs)
    # Macro choice
    aM, compM, GsM, dM = choose_action_planner(Qs, A_coarse, B_coarse, gm.C, horizon=H_macro, obs_prune_eps=prune)
    # Micro refinement (we could bias by macro suggestion; here we just plan normally)
    aF, compF, GsF, dF = choose_action_planner(qs, gm.A, gm.B, gm.C, horizon=H_micro, obs_prune_eps=prune)
    nodes = dM.nodes_expanded + dF.nodes_expanded
    return {
        "a_macro": aM, "a_micro": aF,
        "nodes_total": nodes, "nodes_macro": dM.nodes_expanded, "nodes_micro": dF.nodes_expanded
    }

# Demo
res = hierarchical_plan(gm, qs0, M=max(M_LIST), H_macro=2, H_micro=1, prune=PRUNE)
res

### Compare to full fine-level planning at the larger horizon
We compare the hierarchical nodes vs. a single fine planner at $H=H_{macro}+H_{micro}$ and whether the first action matches the full fine planner’s first action at that horizon (a rough proxy for quality on this toy).

In [ ]:
H_macro, H_micro = 2, 1
M_demo = max(M_LIST)
hier = hierarchical_plan(gm, qs0, M=M_demo, H_macro=H_macro, H_micro=H_micro, prune=PRUNE)
full = bench_fine(gm, qs0, H=H_macro+H_micro, prune=0.0)
print("Hierarchical:", hier)
print("Full fine H=", H_macro+H_micro, ": nodes=", full['nodes'], ", a=", full['a'])
print("First action match? →", (hier['a_macro'] == full['a']) or (hier['a_micro'] == full['a']))

## Takeaways
- Coarse-graining (smaller $M$) **reduces planning nodes/time** for the same horizon $H$.
- On this symmetric ring, the **first action** often agrees across levels even when $M \ll N$.
- A **hierarchical plan** (macro then micro) can approximate a deeper fine plan with far fewer nodes — especially as $H$ grows or when $N$ is large.
- For richer worlds, you’ll want task-specific mappings (macro options) and local micro refinements conditioned on macro suggestions; the same operators ($R,L$) and approach extend naturally.